# `mle-hyperopt`: Lightweight Hyperparameter Optimization
### Author: [@RobertTLange](https://twitter.com/RobertTLange) [Last Update: October 2021] [![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RobertTLange/mle-logging/blob/main/examples/getting_started.ipynb)

In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

#!pip install -q mle-hyperopt

In [2]:
def fake_train(lrate, batch_size, arch):
    """ Optimum: lrate=0.2, batch_size=4, arch='conv'."""
    f1 = ((lrate - 0.2) ** 2 + (batch_size - 4) ** 2
          + (0 if arch == "conv" else 10))
    return f1

## Basic API Usage: Grid Search

Note that the API assumes that we are minimizing an objective. If you want to maximize simply multiply all objective values with minus 1.

In [3]:
from mle_hyperopt import GridSearch
# Instantiate grid search class
strategy = GridSearch(real={"lrate": {"begin": 0.1,
                                      "end": 0.5,
                                      "bins": 5}},
                      integer={"batch_size": {"begin": 1,
                                              "end": 5,
                                              "spacing": 1}},
                      categorical={"arch": ["mlp", "cnn"]})

# Simple ask - eval - tell API
configs = strategy.ask(batch_size=2)
values = [fake_train(**c) for c in configs]

In [4]:
strategy.tell(configs, values)
configs, values

([{'arch': 'mlp', 'batch_size': 1, 'lrate': 0.1},
  {'arch': 'mlp', 'batch_size': 1, 'lrate': 0.2}],
 [19.009999999999998, 19.0])

### Saving and reloading previous search results

In [7]:
# Storing of results to .pkl
strategy.save("search_log.json")

Stored 2 search iterations.


In [9]:
# Reloading of results from .pkl
strategy.load("search_log.json")

[{'eval_id': 0, 'params': {'arch': 'mlp', 'batch_size': 1, 'lrate': 0.1}, 'objective': 19.009999999999998}, {'eval_id': 1, 'params': {'arch': 'mlp', 'batch_size': 1, 'lrate': 0.2}, 'objective': 19.0}]
{'arch': 'mlp', 'batch_size': 1, 'lrate': 0.1} was previously evaluated.
{'arch': 'mlp', 'batch_size': 1, 'lrate': 0.2} was previously evaluated.
Reloaded 0 previous search iterations.


In [ ]:
strategy.log

In [ ]:
strategy = GridSearch(real={"lrate": {"begin": 0.1,
                                      "end": 0.5,
                                      "bins": 5}},
                      integer={"batch_size": {"begin": 1,
                                              "end": 5,
                                              "spacing": 1}},
                      categorical={"arch": ["mlp", "cnn"]},
                      reload_path="search_log.pkl")

### Inspecting the search results

In [ ]:
# Return flattened log as pandas dataframe
strategy.to_df()

In [ ]:
# Retrieving the best performing configuration
strategy.get_best()

In [ ]:
# Plot timeseries of best performing score over search iterations
strategy.plot_best()

### Adding fixed parameters & storing configuration files

In [ ]:
strategy = GridSearch(real={"lrate": {"begin": 0.1,
                                      "end": 0.5,
                                      "bins": 5}},
                      integer={"batch_size": {"begin": 1,
                                              "end": 5,
                                              "spacing": 1}},
                      categorical={"arch": ["mlp", "cnn"]},
                      fixed_params={"momentum": 0.9})
strategy.ask(2, store=True)

## Single-Objective: Random Search

In [ ]:
from mle_hyperopt import RandomSearch

strategy = RandomSearch(real={"lrate": {"begin": 0.1,
                                        "end": 0.5}},
                        integer={"batch_size": {"begin": 1,
                                                "end": 5,
                                                "spacing": 1}},
                        categorical={"arch": ["mlp", "cnn"]},
                        search_config={"refine_after": 5,
                                       "refine_top_k": 2})

configs = strategy.ask(5)
values = [fake_train(**c) for c in configs]
strategy.tell(configs, values)

In [ ]:
strategy.get_best(2)

In [ ]:
strategy.print_ranking(4)

## Single-Objective: Sequential Model-Based Optimization (SMBO)

In [ ]:
from mle_hyperopt import SMBOSearch

strategy = SMBOSearch(real={"lrate": {"begin": 0.1,
                                      "end": 0.5,
                                      "prior": "uniform"}},
                      integer={"batch_size": {"begin": 1,
                                              "end": 5,
                                              "prior": "uniform"}},
                      search_config={"base_estimator": "GP",
                                     "acq_function": "gp_hedge",
                                     "n_initial_points": 5},
                      fixed_params={"arch": "cnn"})

configs = strategy.ask(5)
values = [fake_train(**c) for c in configs]
strategy.tell(configs, values)

In [ ]:
strategy.print_ranking(4)

## Multi-Objective: `nevergrad`

In [ ]:
from mle_hyperopt import NevergradSearch

strategy = NevergradSearch(real={"lrate": {"begin": 0.1,
                                      "end": 0.5,
                                      "prior": "uniform"}},
                           integer={"batch_size": {"begin": 1,
                                                   "end": 5,
                                                   "prior": "uniform"}},
                           search_config={"optimizer": "NGOpt",
                                          "budget_size": 100,
                                          "num_workers": 5},
                           fixed_params={"arch": "cnn"})

configs = strategy.ask(5)
values = [fake_train(**c) for c in configs]
strategy.tell(configs, values)
strategy.print_ranking(4)

In [ ]:
def multi_fake_train(lrate, batch_size, arch):
    # optimal for learning_rate=0.2, batch_size=4, architecture="conv"
    f1 = ((lrate - 0.2) ** 2 + (batch_size - 4) ** 2
          + (0 if arch == "conv" else 10))
    # optimal for learning_rate=0.3, batch_size=2, architecture="mlp"
    f2 = ((lrate - 0.3) ** 2 + (batch_size - 2) ** 2
          + (0 if arch == "mlp" else 5))
    return f1, f2

In [ ]:
strategy = NevergradSearch(real={"lrate": {"begin": 0.1,
                                      "end": 0.5,
                                      "prior": "uniform"}},
                           integer={"batch_size": {"begin": 1,
                                                   "end": 5,
                                                   "prior": "uniform"}},
                           search_config={"optimizer": "NGOpt",
                                          "budget_size": 100,
                                          "num_workers": 5},
                           fixed_params={"arch": "cnn"})

configs = strategy.ask(5)
values = [multi_fake_train(**c) for c in configs]
strategy.tell(configs, values)
strategy.log

In [ ]:
strategy.print_ranking()

## Coordinate-Wise Search

Start scanning one parameter for fixed others and a fixed budget. Afterwards, fix the optimized parameter to best value and go over to next parameter. Repeat until all parameters are done. `search_config` specifies order of parameters and their default. Internally, we run a coordinate-wise grid search.

In [ ]:
from mle_hyperopt import CoordinateSearch
strategy = CoordinateSearch(real={"lrate": {"begin": 0.1,
                                            "end": 0.5,
                                            "bins": 5}},
                            integer={"batch_size": {"begin": 1,
                                                    "end": 5,
                                                    "spacing": "1"}},
                            search_config={"order": ["lrate", "batch_size"],
                                           "defaults": {"lrate": 0.1,
                                                        "batch_size": 3}},
                            fixed_params={"arch": "cnn"})

configs = strategy.ask(5)
configs

In [ ]:
values = [fake_train(**c) for c in configs]
strategy.tell(configs, values)
strategy.log

In [ ]:
configs = strategy.ask(4)
configs

In [ ]:
values = [fake_train(**c) for c in configs]
strategy.tell(configs, values)
strategy.log

In [ ]:
configs = strategy.ask(5)
configs

In [ ]:
strategy.all_evaluated_params

# `hyperopt` decorator - minimal search wrapper

Note: Assumes that function to evaluate directly consumes a configuration dictionary.

In [ ]:
from mle_hyperopt import hyperopt

@hyperopt(strategy_type="grid",
          num_search_iters=25,
          real={"x": {"begin": 0., "end": 0.5, "bins": 5},
                "y": {"begin": 0, "end": 0.5, "bins": 5}})
def circle_objective(config):
    distance = abs((config["x"] ** 2 + config["y"] ** 2))
    return distance

strategy = circle_objective()
strategy.log

In [ ]:
strategy.to_df()